### 편차보정모델 개발

데이터 로드, 결측제거, 변수 선택

In [1]:
import numpy as np
import pandas as pd
from data_split import data_split

import sys
sys.path.insert(0, '../')
from config.global_params import variable_info


# 인풋 준비
nwp_file = "../DAIO/nwp_data_47105"
obs_file = "../DAIO/obs_data_47105"
nwp_data = np.load(nwp_file)
obs_data = np.load(obs_file)
print("="*50, "load data shape")
print(nwp_data.shape)
print(obs_data.shape)


# train([21.01,04, 22.01,04]) / test([23.01,04]) 분할  
class_split = data_split(nwp_data, obs_data)
train_nwp, test_nwp, train_obs, test_obs = class_split.get_split_data()
print("="*50, "split data shape")
print(train_nwp.shape)
print(train_obs.shape)
print(test_nwp.shape)
print(test_obs.shape)



# 결측제거
missing_nwp_train = set(np.where(np.isnan(train_nwp))[0])
missing_obs_train = set(np.where(np.isnan(train_obs))[0])
missing_all_train = list(missing_nwp_train | missing_obs_train)
print("결측 합계: ", len(missing_all_train))
dm_nwp_train = np.delete(train_nwp, missing_all_train, 0)
dm_obs_train = np.delete(train_obs, missing_all_train, 0)
print("shape of after drop")
print(dm_nwp_train.shape)
print(dm_obs_train.shape)

missing_nwp_test = set(np.where(np.isnan(test_nwp))[0])
missing_obs_test = set(np.where(np.isnan(test_obs))[0])
missing_all_test = list(missing_nwp_test | missing_obs_test)
print("결측 합계: ", len(missing_all_test))
dm_nwp_test = np.delete(test_nwp, missing_all_test, 0)
dm_obs_test = np.delete(test_obs, missing_all_test, 0)
print("shape of after drop")
print(dm_nwp_test.shape)
print(dm_obs_test.shape)


# 변수선택
sel_var = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel']
var_list_dict = list(variable_info.keys())
var_index = [ var_list_dict.index(i) for i in sel_var ]
#print(var_list_dict)
#print(var_index)
sel_dm_nwp_train = dm_nwp_train[:,:,var_index]
sel_dm_nwp_test = dm_nwp_test[:,:,var_index]
print("="*50, "drop data shape")
print(sel_dm_nwp_train.shape)
print(dm_obs_train.shape)
print(sel_dm_nwp_test.shape)
print(dm_obs_test.shape)

================================================== load data shape
(868, 49, 20)
(868, 49, 2)
================================================== split data shape
(109, 49, 20)
(109, 49, 2)
(61, 49, 20)
(61, 49, 2)
결측 합계:  1
shape of after drop
(108, 49, 20)
(108, 49, 2)
결측 합계:  5
shape of after drop
(56, 49, 20)
(56, 49, 2)
================================================== drop data shape
(108, 49, 6)
(108, 49, 2)
(56, 49, 6)
(56, 49, 2)


스케일링

In [2]:
# .. 스케일링 및 데이터 분할
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#-------------------------------------------------------------------------
# .. Normalize

output_size = 2

# .. initialaize
tr_b, tr_s, tr_f = sel_dm_nwp_train.shape[0], sel_dm_nwp_train.shape[1], sel_dm_nwp_train.shape[2]      
ts_b, ts_s, ts_f = sel_dm_nwp_test.shape[0], sel_dm_nwp_test.shape[1], sel_dm_nwp_test.shape[2]      

# .. get restorator with obs range
nwp_scaler = MinMaxScaler()   # copy default true
obs_scaler = MinMaxScaler()
nwp_scaler.fit(sel_dm_nwp_train.view().reshape(tr_b*tr_s, tr_f))
obs_scaler.fit(dm_obs_train.view().reshape(tr_b*tr_s, output_size))

# .. feature normalize   ( train seq, feature = test seq, feature )
nor_dm_nwp_train = nwp_scaler.transform(sel_dm_nwp_train.reshape(tr_b*tr_s, tr_f))
nor_dm_nwp_train = nor_dm_nwp_train.reshape(tr_b,tr_s,tr_f)
nor_dm_obs_train = obs_scaler.transform(dm_obs_train.reshape(tr_b*tr_s, output_size))
nor_dm_obs_train = nor_dm_obs_train.reshape(tr_b,tr_s, output_size)

nor_dm_nwp_test = nwp_scaler.transform(sel_dm_nwp_test.reshape(ts_b*ts_s, ts_f))
nor_dm_nwp_test = nor_dm_nwp_test.reshape(ts_b,ts_s,ts_f)
nor_dm_obs_test = obs_scaler.transform(dm_obs_test.reshape(ts_b*ts_s, output_size))
nor_dm_obs_test = nor_dm_obs_test.reshape(ts_b,ts_s, output_size)

nor_dm_nwp_train = nor_dm_nwp_train[:,1::,:]
nor_dm_obs_train = nor_dm_obs_train[:,1::,:]

nor_dm_nwp_test = nor_dm_nwp_test[:,1::,:]
nor_dm_obs_test = nor_dm_obs_test[:,1::,:]

print ('---------- Final training data shape')
print(type(nor_dm_nwp_train))
print ('tran nwp : ', nor_dm_nwp_train.shape)
print ('tran obs : ', nor_dm_obs_train.shape)
print ('test nwp : ', nor_dm_nwp_test.shape)
print ('test obs : ', nor_dm_obs_test.shape)

---------- Final training data shape
<class 'numpy.ndarray'>
tran nwp :  (108, 48, 6)
tran obs :  (108, 48, 2)
test nwp :  (56, 48, 6)
test obs :  (56, 48, 2)


#### 모델 설정

In [16]:
from tensorflow.compat.v1.keras.backend import set_session
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers


from tensorflow.keras.utils import plot_model as plm
from tcn import TCN, tcn_full_summary
from tensorflow.keras.activations import swish

import os
import joblib

#import keras
#from keras.wrappers.scikit_learn import KerasRegressor
#from tensorflow.keras.models import model_from_json
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.keras import Input, Model, callbacks
from keras.callbacks import CSVLogger

#-------------------------------------------------------------------------
# .. Set configure

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


#-------------------------------------------------------------------------
# .. Data set

element = 'ALLV'
name_list = "./SHEL/namelist.input"

hp_lr = 0.009
hp_pd = 'same'
hp_ns = 1
hp_dl = [1,2,4,8,16,32,48]
hp_ldl = hp_dl[-1] # last dilation factor to make name of save model
hp_bn = True
hp_nf = 80
hp_dr = 0.07
hp_ks = 6

input_size = 6
output_size = 2
num_fct = 48
batch_size = 8
n_iter_search = 20
num_epoch = 1000
dev_stn_id = 47105
tran_data_per = "2101_2104_2201_2204"

exp_name = "CNTL"
csv_outdir = './DAOU/LOSS/' + exp_name + '/'
model_outdir = './DAOU/MODL/' + exp_name + '/'
scalr_outdir = './DAOU/SCAL/' + exp_name + '/'
gifd_outdir = './GIFD/' + exp_name + '/'
log_outdir = './DAOU/LOGF/' + exp_name + '/'


if os.path.exists(csv_outdir) != True: os.makedirs(csv_outdir)
if os.path.exists(model_outdir) != True: os.makedirs(model_outdir)
if os.path.exists(scalr_outdir) != True: os.makedirs(scalr_outdir)
if os.path.exists(gifd_outdir) != True: os.makedirs(gifd_outdir)
if os.path.exists(log_outdir) != True: os.makedirs(log_outdir)

#-------------------------------------------------------------------------
# .. Set Model

# .. Set batch for whole data
batch_size = 8

# .. create model - API
i = Input( batch_shape=(None, num_fct, input_size) ) 
o = TCN(return_sequences=True, 
        activation=swish, 
        nb_filters=hp_nf, 
        padding=hp_pd,
        use_batch_norm = hp_bn,
        nb_stacks=hp_ns,
        dropout_rate=hp_dr,
        kernel_size=hp_ks,
        use_skip_connections=True,
        dilations=hp_dl
        )(i)
o = TimeDistributed(Dense(output_size, activation='linear'))(o)

adam = optimizers.Adam(learning_rate=hp_lr)
m= Model(inputs=[i], outputs=[o])
m.compile(optimizer=adam, loss='mse')

#tcn_full_summary(m, expand_residual_blocks=True)
m.summary()
plm(m, to_file='./GIFD/tcn_exp.png', show_shapes=True)


#=========================================================================
# .. Save model


#-------------------------------------------------------------------------
# .. Set model label

make_option = '_e' + str(num_epoch) + '_bs' + str(batch_size) + '_lr' + str(hp_lr) + \
              '_nf' + str(hp_nf) + '_pd' + hp_pd + '_ks' + str(hp_ks) + '_dr'+str(hp_dr) + \
              '_dl' + str(hp_ldl) + '_ns' + str(hp_ns) 
model_name = 'tcn_modl_' + 'var' + str(input_size) + make_option + \
             '_' + tran_data_per + '_' + str(dev_stn_id) + '.h5'
loss_name = 'loss_' + 'var' + str(input_size) + make_option + \
             '_' + tran_data_per + '_' + str(dev_stn_id)
scalr_name = 'sclr_' + 'var' + str(input_size) + make_option + \
             '_' + tran_data_per + '_' + str(dev_stn_id) + '.pkl'
log_name = 'log_' + 'var' + str(input_size) + make_option + \
             '_' + tran_data_per + '_' + str(dev_stn_id) + '.csv'



#-------------------------------------------------------------------------
# .. save scaler
joblib.dump(nwp_scaler, scalr_outdir + 'nwp_' + scalr_name)
joblib.dump(obs_scaler, scalr_outdir + 'obs_' + scalr_name)




Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 6)]           0         
                                                                 
 tcn_2 (TCN)                 (None, 48, 80)            508240    
                                                                 
 time_distributed_2 (TimeDi  (None, 48, 2)             162       
 stributed)                                                      
                                                                 
Total params: 508402 (1.94 MB)
Trainable params: 506162 (1.93 MB)
Non-trainable params: 2240 (8.75 KB)
_________________________________________________________________


['./DAOU/SCAL/CNTL/obs_sclr_var6_e1000_bs8_lr0.009_nf80_pdsame_ks6_dr0.07_dl48_ns1_2101_2104_2201_2204_47105.pkl']

In [15]:
#-------------------------------------------------------------------------
# .. training model


callbacks_list = [ callbacks.ModelCheckpoint(filepath=model_outdir + model_name, monitor='val_loss', save_best_only=True, verbose=1) ]
csv_logger = CSVLogger(log_outdir + log_name, append=True, separator=';')

hist = m.fit(nor_dm_nwp_train, nor_dm_obs_train,
      epochs=num_epoch,
      batch_size=batch_size, 
      callbacks=callbacks_list,
      shuffle=True)
      #validation_data=(nor_vald_x, nor_vald_y) )


#-------------------------------------------------------------------------
# .. Save Loss history of best model

# loss_df = pd.DataFrame(list(zip(hist.history['loss'],hist.history['val_loss'])),
#                        columns=["tran","eval"])
loss_df = pd.DataFrame(list(hist.history['loss']),
                       columns=["tran"])
loss_df.to_csv(csv_outdir+loss_name)



#-------------------------------------------------------------------------
# .. Clear model

K.clear_session()

Epoch 1/1000
13/14 [==========================>...] - ETA: 0s - loss: 5.1768WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 7s 26ms/step - loss: 5.0311
Epoch 2/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.9502WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.9409
Epoch 3/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.5178WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.5161
Epoch 4/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.3025WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.2992
Epoch 5/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.2519WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.2482
Epoch 6/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.2077WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.2098
Epoch 7/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.1905WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.1882
Epoch 8/1000
13/14 [==========================>...] - ETA: 0s - loss: 0.1562WARNING:tensorflow:Can save best model only with val_loss available, skipping.


14/14 [==============================] - 0s 26ms/step - loss: 0.1534
Epoch 9/1000
 3/14 [=====>........................] - ETA: 0s - loss: 0.2623

KeyboardInterrupt: 